# pyBela - Logger


### Log for a fixed period of time

In [ ]:
from pyBela import Logger

# setup
logger = Logger(project_name="logger-example", # name of the Bela project
                file_name="study_1.log", # name of the log file (created in the Bela project folder)
                time_ms=60*1000) # time in ms to log for (default is 1 min)
"""
 <--- python sends a message to Bela to start logging:
{
    "command": "start_logging",
    "project_name": "logger-example",
    "file_name": "study_1.log",
}
 """ 

# python asynchronously waits for Bela to send a confirmation message
"""
---> Bela sends a confirmation message:
{   
    "status_code": 1,
    "message": "Logging started..." ,
    "time_ms": 60000
    "variables": ["analogFramesElapsed", "sensorValue"]
    "types": ["int", "float"]
}
"""
# print: Logging started...

# after 1 min... (timing is managed in Bela)
# print: Logging finished
"""
---> Bela sends a confirmation message:
{   
    "status_code": 1,
    "message": "Logging finished" 
}
"""

### Log until stopped / interrupted

In [ ]:
from pyBela import Logger

# setup
logger = Logger(project_name="logger-example", # name of the Bela project
                file_name="study_1.log", # name of the log file (created in the Bela project folder)
                time_ms=0) # time in ms to log for  -- 0 means it will log indefinitely

# python asynchronously waits for Bela to send a confirmation message
"""
---> Bela sends a confirmation message:
{   
    "status_code": 1,
    "message": "Logging started...",
    "file_name": "study_1.log",
    "time_ms": 0, # 0 means it will log indefinitely
    "logged_variables": ["var1", "var2"]
    "types": ["int", "float"]
}
# logged variables and types are stored in the logger object (required for later decoding)
"""
# print: Logging started...
# if the Bela program is interrupted, or if the logging is finished from the python side (see cell below)
# print: Logging interrupted

In [ ]:
logger.stop() # stop logging
# python asynchronously waits for Bela to send a confirmation message
"""
 <--- python sends a message to Bela to stop logging:
{
    "command": "stop_logging",
    "project_name": "logger-example"
}

---> Bela sends a confirmation message:
{   
    "status_code": 1,
    "message": "Logging stopped" 
}
"""
# print: Logging finished

The Bela program runs irrespective of the python side (i.e., stopping the logger won't stop the Bela program). After a logging session is finished, another one can be created with a different file name and logging time.

### Check available space

In [ ]:
logger.check_space(30*1000*60) # check if there is enough space to log for n ms  

"""
 <--- python sends a message to Bela 
{
    "command": "space_available",
    "time_ms": 1800000
}

 ---> Bela sends a message:
{
    "message": "space available for 1800000 ms of logging 2 channels",
}

"""



### Copying logging files back to the host

In [ ]:
# transfer logs to host (for now using ssh)
logger.transfer_logs_to_host() 

#python asynchronously waits for Bela to send a confirmation message
#print: Transferring logs to host...

"""
---> Bela sends a confirmation message:
{
    "status_code": 1,
    "message": "Logs transferred to host"
}
"""
# print: Logs transferred to host

In [ ]:
logger.clean() # clean session log file on the Bela side

#python asynchronously waits for Bela to send a confirmation message

"""
---> Bela sends a confirmation message:
{
    "status_code": 1,
    "message": "Log file deleted on Bela"
}
"""


In [ ]:
logger_data = logger.load() # load the log file and decode it into a python dictionary. Logger knows the number of channels and type of data for each channel (see message sent by Bela when logging starts)